# Feature-Aware Data Augmentation

Augmentation policies to consider:
- Simple policy randomly sampling of n transforms 
- Constrained sampling policy with a blacklist of transforms to avoid 
- Feature-aware augmentation policy where transforms are picked based on their (transform, feature) behavior

In [1]:
# data
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm
from datasets import load_dataset, load_from_disk
from datasets.utils.logging import disable_progress_bar

# amrs
import amrlib
import penman

# transform
# import sibyl
import torch
import inspect
import random
from functools import partial

# eval pipeline
import pandas as pd
from transformers import pipeline
from huggingface_hub import HfApi, ModelFilter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from scipy.special import softmax

# train pipeline
import shutil
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    Trainer, 
    TrainingArguments, 
    EarlyStoppingCallback
)

# cleanlab pipeline
from cleanlab.filter import find_label_issues

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from nlaugmenter.transformations.emojify                                   import EmojifyTransformation
from nlaugmenter.transformations.synonym_insertion                         import SynonymInsertion
from nlaugmenter.transformations.back_translation                          import BackTranslation
from nlaugmenter.transformations.noun_compound_paraphraser                 import NounCompoundParaphraser
from nlaugmenter.transformations.diacritic_removal                         import DiacriticRemoval
from nlaugmenter.transformations.azerty_qwerty_chars_swap                  import AzertyQwertyCharsSwap
from nlaugmenter.transformations.add_hashtags                              import HashtagGeneration
from nlaugmenter.transformations.replace_spelling                          import SpellingTransformation
from nlaugmenter.transformations.token_replacement                         import TokenReplacement
from nlaugmenter.transformations.auxiliary_negation_removal                import SentenceAuxiliaryNegationRemoval
from nlaugmenter.transformations.use_acronyms                              import UseAcronyms
from nlaugmenter.transformations.yoda_transform                            import YodaPerturbation
from nlaugmenter.transformations.factive_verb_transformation               import FactiveVerbTransformation
from nlaugmenter.transformations.protaugment_diverse_paraphrase            import ProtaugmentDiverseParaphrase
from nlaugmenter.transformations.replace_financial_amounts                 import ReplaceFinancialAmount
from nlaugmenter.transformations.americanize_britishize_english            import AmericanizeBritishizeEnglish
from nlaugmenter.transformations.punctuation                               import PunctuationWithRules
from nlaugmenter.transformations.urban_dict_swap                           import UrbanThesaurusSwap
from nlaugmenter.transformations.filler_word_augmentation                  import FillerWordAugmentation
from nlaugmenter.transformations.synonym_substitution                      import SynonymSubstitution
from nlaugmenter.transformations.style_paraphraser                         import StyleTransferParaphraser
from nlaugmenter.transformations.random_upper_transformation               import RandomUpperPerturbation
from nlaugmenter.transformations.weekday_month_abbreviation                import WeekdayMonthAbbreviation
from nlaugmenter.transformations.city_names_transformation                 import CityNamesTransformation
from nlaugmenter.transformations.sentence_additions                        import SentenceAdditions
from nlaugmenter.transformations.change_date_format                        import ChangeDateFormat
from nlaugmenter.transformations.tense                                     import TenseTransformation
from nlaugmenter.transformations.dyslexia_words_swap                       import DyslexiaWordsSwap
from nlaugmenter.transformations.gender_culture_diverse_name               import GenderCultureDiverseName
from nlaugmenter.transformations.gender_neutral_rewrite                    import GenderNeutralRewrite
from nlaugmenter.transformations.lost_in_translation                       import LostInTranslation
from nlaugmenter.transformations.insert_abbreviation                       import AbbreviationInsertionEN
from nlaugmenter.transformations.random_deletion                           import RandomDeletion
from nlaugmenter.transformations.transformer_fill                          import TransformerFill
from nlaugmenter.transformations.concat_monolingual                        import ConcatMonolingual
from nlaugmenter.transformations.country_state_abbreviation_transformation import CountryStateAbbreviation
from nlaugmenter.transformations.butter_fingers_perturbation               import ButterFingersPerturbation
from nlaugmenter.transformations.replace_abbreviation_and_acronyms         import ReplaceAbbreviations
from nlaugmenter.transformations.slangificator                             import Slangificator
from nlaugmenter.transformations.yes_no_question                           import YesNoQuestionPerturbation
from nlaugmenter.transformations.gender_swap                               import GenderSwap
from nlaugmenter.transformations.close_homophones_swap                     import CloseHomophonesSwap
from nlaugmenter.transformations.simple_ciphers                            import SimpleCiphers
from nlaugmenter.transformations.change_person_named_entities              import ChangePersonNamedEntities
from nlaugmenter.transformations.greetings_and_farewells                   import GreetingsAndFarewells
from nlaugmenter.transformations.discourse_marker_substitution             import DiscourseMarkerSubstitution
from nlaugmenter.transformations.summarization_transformation              import Summarization
from nlaugmenter.transformations.visual_attack_letters                     import VisualAttackLetters
from nlaugmenter.transformations.sentence_reordering                       import SentenceReordering
from nlaugmenter.transformations.change_char_case                          import ChangeCharCase
from nlaugmenter.transformations.antonyms_substitute                       import AntonymsSubstitute
from nlaugmenter.transformations.mix_transliteration                       import MixTransliteration
from nlaugmenter.transformations.disability_transformation                 import DifferentAbilityTransformation
from nlaugmenter.transformations.abbreviation_transformation               import Abbreviate
from nlaugmenter.transformations.replace_with_hyponyms_hypernyms           import ReplaceHypernyms
from nlaugmenter.transformations.replace_with_hyponyms_hypernyms           import ReplaceHyponyms
from nlaugmenter.transformations.grapheme_to_phoneme_transformation        import PhonemeSubstitution
from nlaugmenter.transformations.multilingual_lexicon_perturbation         import MultilingualLexiconPerturbation
from nlaugmenter.transformations.multilingual_back_translation             import MultilingualBackTranslation
from nlaugmenter.transformations.unit_converter                            import UnitConverter
from nlaugmenter.transformations.adjectives_antonyms_switch                import SentenceAdjectivesAntonymsSwitch
from nlaugmenter.transformations.correct_common_misspellings               import CorrectCommonMisspellings
from nlaugmenter.transformations.neural_question_paraphraser               import NeuralParaphaserPerturbation
from nlaugmenter.transformations.subject_object_switch                     import SentenceSubjectObjectSwitch
from nlaugmenter.transformations.numeric_to_word                           import NumericToWord
from nlaugmenter.transformations.multilingual_dictionary_based_code_switch import MultilingualDictionaryBasedCodeSwitch
from nlaugmenter.transformations.diverse_paraphrase                        import DiverseParaphrase
from nlaugmenter.transformations.english_inflectional_variation            import EnglishInflectionalVariation
from nlaugmenter.transformations.replace_numerical_values                  import ReplaceNumericalValues
from nlaugmenter.transformations.speech_disfluency_perturbation            import SpeechDisfluencyPerturbation
from nlaugmenter.transformations.whitespace_perturbation                   import WhitespacePerturbation
from nlaugmenter.transformations.contraction_expansions                    import ContractionExpansions
from nlaugmenter.transformations.color_transformation                      import ColorTransformation
from nlaugmenter.transformations.contextual_meaning_perturbation           import ContextualMeaningPerturbation
from nlaugmenter.transformations.pig_latin                                 import PigLatin
from nlaugmenter.transformations.leet_letters                              import LeetLetters
from nlaugmenter.transformations.hashtagify                                import HashtagifyTransformation
from nlaugmenter.transformations.propbank_srl_roles                        import CheckSrl
from nlaugmenter.transformations.geo_names_transformation                  import GeoNamesTransformation

C:\Users\fabri\anaconda3\envs\fada\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fabri\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
torch.use_deterministic_algorithms(False)
disable_progress_bar()

In [6]:
# helper functions

def normalize_minmax(df):
    for column in df.columns:
        df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())  
    return df

def normalize_sum(df):
    for column in df.columns:
        df[column] = df[column] / df[column].sum()
    return df

def augment_data(batch, transform, keep_originals=True):
    new_texts, new_labels = [], []
    for text, label in zip(batch['text'], batch['label']):
        new_text, new_label = transform.apply([text], [label])
        new_texts.extend(new_text)
        new_labels.extend(new_label)
    if keep_originals:
        return {"text": batch['text'] + new_texts, "label": batch['label'] + new_labels}
    else:
        return {"text": new_texts, "label": new_labels}
    
def percent_dataset_changed(d1, d2):
    return sum([t1['text'] != t2['text'] for t1, t2 in zip(d1, d2)]) / len(d1) 

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    acc = accuracy_score(labels, predictions.argmax(-1))
    precision, recall, fbeta_score, support = precision_recall_fscore_support(
        y_true=labels, 
        y_pred=predictions.argmax(-1), 
        average="weighted", 
        zero_division=0)
    return { 'accuracy': acc , 
             'precision': precision, 
             'recall': recall, 
             'fbeta_score': fbeta_score} 

def compute_accuracy(predictions, labels):
    if not isinstance(labels, np.ndarray):
        labels = np.array(labels)
    if len(labels.shape) > 1:
        acc = acc_at_k(labels, predictions, k=2)       
    else:
        acc = accuracy_score(labels, np.argmax(predictions, -1))
    return acc

def vectorize(output):
    sorted_output = sorted(output, key=lambda d: d['label']) 
    probs = torch.tensor([d['score'] for d in sorted_output])
    return probs

def sample_transforms(transforms, p, n=2, replace=False):
    return np.random.choice(transforms, size=n, p=p, replace=replace).tolist()

def policy_heatmap(policy, transforms, featurizers):
    t_names = [t.transform_class.__name__ for t in transforms]
    f_names = [f.__name__ for f in featurizers]
    df = pd.DataFrame(policy)
    df.columns = f_names
    df.index = t_names
    sns.heatmap(df)
    plt.show()
    
def implement_policy_probabilities(policy, features):
    default_probability = policy.mean(axis=1)
    policy_probs = []
    for f in features:
        available_features = np.nonzero(f)[0]
        if len(available_features) == 0:
            probs = default_probability
        else:
            probs = policy[:, available_features].mean(axis=1)
        policy_probs.append(probs)
    return np.array(policy_probs)

In [7]:
class Transform:
    def __init__(self, transform_class, num_outputs=1, task_name="sentiment"):
        self.transform_class = transform_class
        self.num_outputs = num_outputs
        self.task_name = task_name
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.intakes_target = False
        self.is_batched = False
        
        # setting class attributes
        if 'to_tense' in inspect.signature(self.transform_class).parameters:
            print(f"initializing {self.transform_class.__name__} with to_tense='past'") # future & random don't work
            self.transform_instance = self.transform_class(to_tense="past")
        elif 'source_lang' in inspect.signature(self.transform_class).parameters:
            print(f"initializing {self.transform_class.__name__} with source_lang='es'") 
            self.transform_instance = self.transform_class(source_lang="es")
        elif 'task_name' in inspect.signature(self.transform_class).parameters:
            print(f"initializing {self.transform_class.__name__} with task_name='{task_name}', return_metadata=True") 
            self.transform_instance = self.transform_class(task_name=self.task_name, return_metadata=True)
        elif isinstance(self.transform_class, LostInTranslation):
            print(f"initializing {self.transform_class.__name__} with device=0")
            self.transform_instance = self.transform_class(device=0)
        else:
            print(f"initializing {self.transform_class.__name__}")
            self.transform_instance = self.transform_class()
        
        # setting instance attributes
        if hasattr(self.transform_instance, "max_outputs"):
            print(f"setting max_outputs={self.num_outputs}")
            self.transform_instance.max_outputs = self.num_outputs
        if hasattr(self.transform_instance, "max_paraphrases"):
            print(f"setting max_paraphrases={self.num_outputs}")
            self.transform_instance.max_paraphrases = self.num_outputs
        if hasattr(self.transform_instance, "device"):
            if self.transform_instance.device is None or self.transform_instance.device == 'cpu':
                print(f"setting device={self.device}")
                self.transform_instance.device = self.device
        
        # selecting the transformation function
        if hasattr(self.transform_class, "generate"):
            self.transform_fn = self.transform_instance.generate
        if hasattr(self.transform_class, "augment"):
            self.transform_fn = self.transform_instance.augment
        if hasattr(self.transform_class, "transform_batch"):
            self.transform_fn = self.transform_instance.transform_batch
            self.intakes_target = True
            self.is_batched = True
            
    def synced_shuffle(self, list1, list2):
        # Shuffle two lists with same order
        # Using zip() + * operator + shuffle()
        temp = list(zip(list1, list2))
        random.shuffle(temp)
        res1, res2 = zip(*temp)
        # res1 and res2 come out as tuples, and so must be converted to lists.
        res1, res2 = list(res1), list(res2)
        return res1, res2
            
    def apply(self, texts, labels=None):
        if self.intakes_target:
            if self.is_batched:
                new_texts, new_labels = self.transform_fn((texts, labels))
            else:
                new_texts, new_labels = [], []
                for t, l in zip(texts, labels):
                    new_t, new_l = self.transform_fn(t, l)
                    new_texts.append(new_t)
                    new_labels.extend([new_l] * len(new_t))
        else:
            if self.is_batched:
                new_texts = self.transform_fn((texts))
                new_texts = labels
            else:
                new_texts, new_labels = [], []
                for t, l in zip(texts, labels):
                    new_t = self.transform_fn(t)
                    if len(new_t) > self.num_outputs:
                        new_t = new_t[:self.num_outputs]
                    new_texts.extend(new_t)
                    new_labels.extend([l] * len(new_t))
                    
        # post processing since some transformations add/remove more new outputs than expected
        if len(new_texts) == 0:
            print("no new_texts, substituting original texts...")
            new_texts = texts
        if len(new_labels) == 0:
            print("no new_labels, substituting original labels...")
            new_labels = labels
        new_texts, new_labels = self.synced_shuffle(new_texts, new_labels)
        
        expected_len = len(texts) * self.num_outputs
        new_texts = new_texts[:expected_len]
        new_labels = new_labels[:expected_len]
        
        return new_texts, new_labels

In [8]:
class AMRGraph:
    def __init__(self, amr):
        self.graph = penman.decode(amr) if not isinstance(amr, penman.graph.Graph) else amr
        self.amr_text = penman.encode(self.graph)

    def contains_concept(self, concepts):
        """
        Concepts are nodes / instances in the AMR graph.
        """
        try:
            if not isinstance(concepts, list): concepts = [concepts]
            graph_concepts = [t.target for t in self.graph.instances()]
            return any(c for c in graph_concepts if c in concepts)
        except Exception as e:
            print(e)
            print(self.graph, self.amr_text)

    def contains_role(self, roles):
        """
        Roles are edges in the AMR graph.
        """
        try:
            if not isinstance(roles, list): roles = [roles]
            graph_roles = [e.role for e in self.graph.edges()]
            return any(r for r in graph_roles if r in roles)
        except Exception as e:
            print(e)
            print(self.graph, self.amr_text)

    def contains_attribute(self, attributes):
        """
        Attributes are properties of concept nodes, i.e. relationships to 
        constant values.
        """
        try:
            if not isinstance(attributes, list): attributes = [attributes]
            graph_attrs = [a.target for a in self.graph.attributes()]
            return any(a for a in graph_attrs if a in attributes)
        except Exception as e:
            print(e)
            print(self.graph, self.amr_text)

In [9]:
# attributes =============================================================

def contains_imperative(g): return g.contains_attribute("imperative")
def contains_exlamation(g): return g.contains_attribute("expressive")
def contains_negation(g):   return g.contains_attribute("-")

# concepts ===============================================================

def contains_conjunctions(g):         return g.contains_concept(["and", "or", "contrast-01", "either", "neither"])
def contains_interrogative_clause(g): return g.contains_concept("truth-value")
def contains_question(g):             return g.contains_concept(["amr-unknown", "amr-choice"])

# roles ==================================================================

def contains_coreferences(g): return any(r for r in g.amr_text.split() if r in ['i', 'you', 'he', 'she', 'it', 'we', 'they'])
def contains_number(g):       return any(a for a in g.graph.attributes() if a.target.isnumeric())

def contains_accompanier(g):  return g.contains_role(':accompanier')
def contains_age(g):          return g.contains_role(':age')
def contains_beneficiary(g):  return g.contains_role(':beneficiary')
def contains_concession(g):   return g.contains_role(':concession')
def contains_condition(g):    return g.contains_role(':condition')
def contains_consist_of(g):   return any(r for r in g.amr_text.split() if r in [':consist-of'])
def contains_degree(g):       return g.contains_role(':degree')
def contains_destination(g):  return g.contains_role(':destination')
def contains_direction(g):    return g.contains_role(':direction')
def contains_domain(g):       return g.contains_role(':domain')
def contains_duration(g):     return g.contains_role(':duration')
def contains_example(g):      return g.contains_role(':example')
def contains_extent(g):       return g.contains_role(':extent')
def contains_frequency(g):    return g.contains_role(':frequency')
def contains_instrument(g):   return g.contains_role(':instrument')
# def contains_li(g):           return g.contains_role(':li')
def contains_location(g):     return g.contains_role(':location')
def contains_manner(g):       return g.contains_role(':manner')
def contains_medium(g):       return g.contains_role(':medium')
def contains_mod(g):          return g.contains_role(':mod')
def contains_mode(g):         return any(a for a in g.graph.attributes() if ":mode" in a.role)
def contains_name(g):         return g.contains_role(':name')
def contains_ord(g):          return g.contains_role(':ord')
def contains_part(g):         return g.contains_role(':part')
def contains_path(g):         return g.contains_role(':path')
def contains_polarity(g):     return g.contains_role(':polarity')
def contains_polite(g):       return any(r for r in g.amr_text.split() if r in [':polite'])
def contains_poss(g):         return g.contains_role(':poss')
def contains_purpose(g):      return g.contains_role(':purpose')
def contains_quant(g):        return g.contains_role(':quant')
def contains_range(g):        return g.contains_role(':range')
def contains_scale(g):        return g.contains_role(':scale')
def contains_source(g):       return g.contains_role(':source')
def contains_subevent(g):     return g.contains_role(':subevent')
def contains_time(g):         return g.contains_role(':time')
def contains_topic(g):        return g.contains_role(':topic')
def contains_unit(g):         return g.contains_role(':unit')
# def contains_value(g):        return g.contains_role(':value')
def contains_wiki(g):         return g.contains_role(':wiki')

In [10]:
class AMRFeatureExtractor:
    
    def __init__(self):
        self.featurizers = featurizers = [    
            contains_accompanier,
            contains_age,
            contains_beneficiary,
            contains_concession,
            contains_condition,
            contains_conjunctions,
            contains_consist_of,
            contains_coreferences,
            contains_degree,
            contains_destination,
            contains_direction,
            contains_domain,
            contains_duration,
            contains_example,
            contains_exlamation,
            contains_extent,
            contains_frequency,
            contains_imperative,
            contains_instrument,
            contains_interrogative_clause,
            contains_location,
            contains_manner,
            contains_medium,
            contains_mod,
            contains_mode,
            contains_name,
            contains_negation,
            contains_number,
            contains_ord,
            contains_part,
            contains_path,
            contains_polarity,
            contains_polite,
            contains_poss,
            contains_purpose,
            contains_quant,
            contains_question,
            contains_range,
            contains_scale,
            contains_source,
            contains_subevent,
            contains_time,
            contains_topic,
            contains_unit
        ]
        self.featurizers = sorted(featurizers, key=lambda f: f.__name__)
        self.amr_model   = None
        
    def load_amr_model(self, max_sent_len=128):
        self.amr_model = amrlib.load_stog_model(max_sent_len=max_sent_len)
        
    def text_to_amr(self, texts):
        if self.amr_model is None:
            self.load_amr_model()
        amr_penmans = self.amr_model.parse_sents(texts, add_metadata=False, disable_progress=True)
        amr_graphs = []
        for p in amr_penmans:
            try:
                amr_graphs.append(AMRGraph(p))
            except Exception as e: 
                print(e)
                print(p)
                amr_graphs.append(AMRGraph(p))
        return amr_graphs
    
    def generate_feature_matrix(self, graphs):
        feature_matrix = []
        for g in graphs:
            feature_vector = []
            for f in self.featurizers:
                feature_vector.append(f(g))
            feature_matrix.append(feature_vector)
        feature_matrix = np.array(feature_matrix, dtype=np.int32)
        return feature_matrix
    
    def __call__(self, texts):
        graphs = self.text_to_amr(texts)
        return self.generate_feature_matrix(graphs)
        

In [11]:
def transforms_to_ids(sampled_transforms, all_transforms):
    transforms_ids = [all_transforms.index(i) for i in sampled_transforms]
    transforms_applied = np.zeros(len(all_transforms), dtype=np.int32)
    transforms_applied[transforms_ids] = 1
    return transforms_applied

In [12]:
class Augmenter:
    def __init__(self, 
                 dataset,
                 transforms,
                 transform_probabilities = None, 
                 num_augmentations_per_record = 5,
                 num_transforms_to_apply = 2,
                 batch_size = 10,
                 allow_resampling = False,
                 keep_originals = False,
                 feature_extractor = None,
                 perf_extractor = None):
        
        self.dataset = dataset
        self.transforms = transforms
        self.transform_probabilities = transform_probabilities
        self.num_augmentations_per_record = num_augmentations_per_record
        self.num_transforms_to_apply = num_transforms_to_apply
        self.batch_size = batch_size
        self.allow_resampling = allow_resampling
        self.keep_originals = keep_originals
        self.feature_extractor = feature_extractor
        self.perf_extractor = perf_extractor
        
        # initializations
        self.dataset = dataset.remove_columns("idx")
        self.add_idx_to_dataset()
        self.num_transforms_available = len(self.transforms)
        
        if self.transform_probabilities is None:
            # set to uniform
            num_examples = len(self.dataset)
            num_transforms = len(self.transforms)
            uniform_policy = np.full((num_examples, num_transforms), fill_value=1/num_transforms)
            self.transform_probabilities = uniform_policy
        
    def add_idx_to_dataset(self):
        if 'idx' not in self.dataset.features:
            self.dataset = self.dataset.add_column("idx", range(len(self.dataset)))
        
    def apply_to_batch(self, batch):
        new_texts, new_labels, transforms_applied, is_changed = [], [], [], []
        for idx, text, label in zip(batch['idx'], batch['text'], batch['label']):
            actual_batch_size = len(batch['idx'])
            original_text, original_label = text, label
            for _ in range(self.num_augmentations_per_record):
                sampled_transforms = sample_transforms(self.transforms, 
                                                       p=self.transform_probabilities[idx], 
                                                       n=self.num_transforms_to_apply, 
                                                       replace=self.allow_resampling)
                transforms_applied.append(transforms_to_ids(sampled_transforms, self.transforms))
                for t in sampled_transforms:
                    try:
                        text, label = t.apply([text], [label])
                        text, label = text[0], label[0]
                    except Exception as e: 
                        print(e)
                        print(f"[Augmenter]: skipping augmentation from {t.transform_class.__name__} on text:'{text}' and label: {label}")

                # avoid adding records with empty text
                if text:
                    new_texts.append(text)
                    new_labels.append(label)
                    is_changed.append(int(original_text != text))

        if self.keep_originals:
            new_texts = batch['text'] + new_texts
            new_labels = batch['label'] + new_labels
            realized_batch_size = len(new_labels)
            transforms_applied = transforms_applied + np.zeros((actual_batch_size, len(self.transforms)), dtype=np.int32).tolist()
            is_changed = is_changed + [0] * actual_batch_size
            out = {
                "text": new_texts, 
                "label": new_labels,
                "idx": list(range(realized_batch_size)),
                "transforms_applied": [t for t in transforms_applied],
                "is_changed": is_changed
            }
        else:
            out = {
                "text": new_texts, 
                "label": new_labels, 
                "idx": list(range(len(new_labels))),
                "transforms_applied": transforms_applied,
                "is_changed": is_changed
            }

        return out
            
                                                   
    def augment(self):
        dataset = self.dataset.map(self.apply_to_batch, batched=True, batch_size=self.batch_size)
        dataset = dataset.remove_columns("idx")
    
        # feature extraction
        if self.feature_extractor is not None:
            features = self.feature_extractor(dataset["text"])
            dataset = dataset.add_column("features", [f for f in features])
                
        # performance scoring    
        if self.perf_extractor is not None:
            performances = self.perf_extractor(dataset["text"], dataset["label"])
            dataset = dataset.add_column("performance", [p for p in performances])
                
        return dataset

In [13]:
class Likelihood:
    def __init__(self):
        self.scorer = torch.nn.NLLLoss(reduction="none")
    
    def __call__(self, probs, targets, indices=None):
        return -self.scorer(probs, targets).numpy()
    
class InverseLikelihood:
    def __init__(self):
        self.scorer = torch.nn.NLLLoss(reduction="none")
    
    def __call__(self, probs, targets, indices=None):
        return 1+self.scorer(probs, targets).numpy()
    
class CleanLabSafe:
    def __init__(self):
        pass
        
    def __call__(self, probs, targets, indices=None):
        probs = probs.numpy()
        targets = targets.numpy()
        scores = ~find_label_issues(
            labels=targets,
            pred_probs=probs,
            n_jobs=1
        )
        return scores.astype(np.int32).tolist()
    
class LikelihoodShift:
    def __init__(self, original_dataset, direction="positive"):
        self.original_dataset = original_dataset
        self.direction = direction
        self.scorer = torch.nn.NLLLoss(reduction="none")
        
    def __call__(self, probs, targets, indices=None):
        new_scores  = -self.scorer(probs, targets).numpy()
        
        old_probs   = torch.tensor(self.original_dataset.select(indices)["preds"])
        old_targets = torch.tensor(self.original_dataset.select(indices)['label'])
        old_scores  = -self.scorer(old_probs, old_targets).numpy()
            
        if self.direction in "positive":
            scores = (new_scores - old_scores).clip(0, 1)
        elif self.direction in "negative":
            scores = (old_scores - new_scores).clip(0, 1)
        else:
            scores = new_scores
        return scores
        
class PerformanceExtractor:
    def __init__(self, dataset_name, scorer, model_id=None):
        self.dataset_name = dataset_name
        self.scorer = scorer
        self.model_id = model_id
        self.api = HfApi()
        self.pipe = None
        self.device = 0 if torch.cuda.is_available() else -1

        if self.model_id and not self.pipe:
            self.create_pipe(self.model_id)

        if not self.pipe:
            self.find_model_for_dataset()

    def create_pipe(self, model_id):
        self.pipe = pipeline("text-classification", 
                            model=model_id, 
                            device=self.device, 
                            padding=True, 
                            truncation=True,
                            top_k=None)
        return self.pipe

    def find_model_for_dataset(self):
        model_filter = ModelFilter(
            task="text-classification",
            library="pytorch",
            # model_name=dataset_name,
            trained_dataset=self.dataset_name)
        model_id = next(iter(self.api.list_models(filter=model_filter)))
        if model_id:
            model_id = getattr(model_id, 'modelId')
            print('Using ' + model_id + ' to support evaluation.')
            self.create_pipe(model_id)

    def extract_prediction_probabilities(self, inputs):
        output = self.pipe(inputs)
        return torch.stack([vectorize(o) for o in output])
    
    def extract_prediction_classes(self, inputs):
        return torch.argmax(self.extract_prediction_probabilities(inputs), axis=1)

    def __call__(self, inputs, targets, indices=None):
        probs   = self.extract_prediction_probabilities(inputs)
        targets = torch.tensor(targets)
        return self.scorer(probs, targets, indices)

## FADA v2.1

In [14]:
dataset_config = ("glue", "sst2")
task_name = "sentiment"

In [15]:
dataset = load_dataset(*dataset_config, split="train")
dataset = dataset.rename_column("sentence", "text")

2023-02-21 19:49:00,103 | datasets.builder | WARNING | Found cached dataset glue (C:/Users/fabri/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [16]:
nlaug_transforms = [
    # EmojifyTransformation, # UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 220: character maps to <undefined>
    SynonymInsertion,
    BackTranslation, # runtime too long (does not attempt to use gpu) (50.15s/ba w gpu)
    # NounCompoundParaphraser, # does not return anything
    DiacriticRemoval,
    AzertyQwertyCharsSwap,
    HashtagGeneration,
    SpellingTransformation,
    TokenReplacement,
    SentenceAuxiliaryNegationRemoval,
    UseAcronyms,
    YodaPerturbation,
    # FactiveVerbTransformation, # cannot find gender.male.names
    ProtaugmentDiverseParaphrase, # somewhat long runtime (29.63s/ba)
    ReplaceFinancialAmount,
    AmericanizeBritishizeEnglish,
    # PunctuationWithRules,  # runtime too long (does not appear to be gpu compatible)
    # UrbanThesaurusSwap, # runtime too long (195.74s/ba)
    FillerWordAugmentation,
    SynonymSubstitution,
    # StyleTransferParaphraser, # runtime too long (192.32s/ba)
    RandomUpperPerturbation,
    WeekdayMonthAbbreviation,
    CityNamesTransformation,
    # SentenceAdditions, # runtime too long (does not attempt to use gpu) # RuntimeError: CUDA out of memory. even at batch_size=1
    ChangeDateFormat,
    TenseTransformation, # index out of range errors / NoneType errors if not using to_tense='past'
    DyslexiaWordsSwap,
    # GenderCultureDiverseName, # does not return anything
    # GenderNeutralRewrite, # index out of range errors
    # LostInTranslation, # IndexError: index out of range in self
    AbbreviationInsertionEN,
    RandomDeletion,
    TransformerFill, # (3.20s/ba)
    ConcatMonolingual,
    CountryStateAbbreviation,
    ButterFingersPerturbation,
    ReplaceAbbreviations, # (4.52s/ba)
    Slangificator,
    YesNoQuestionPerturbation,
    GenderSwap,
    # CloseHomophonesSwap, # runs slow, no gpu (221.04s/ba)
    SimpleCiphers,
    # ChangePersonNamedEntities, # does not return anything
    GreetingsAndFarewells,
    DiscourseMarkerSubstitution,
    Summarization,
    VisualAttackLetters,
    # SentenceReordering, # somwhat slow (41.14s/ba)
    ChangeCharCase,
    AntonymsSubstitute,
    # MixTransliteration, # IndexError: list index out of range
    # DifferentAbilityTransformation, UnboundLocalError: local variable 'text' referenced before assignment
    Abbreviate,
    # ReplaceHypernyms, # RuntimeError: CUDA out of memory. even at batch_size=1
    # ReplaceHyponyms, # RuntimeError: CUDA out of memory. even at batch_size=1
    # PhonemeSubstitution, # IndexError: list index out of range
    # MultilingualLexiconPerturbation, FileNotFoundError: [Errno 2] No such file or directory: '/multilingual_lexicon_uncased.xz'
    MultilingualBackTranslation, # runtime too long (does not attempt to use gpu) (98.04s/ba w gpu)
    UnitConverter,
    SentenceAdjectivesAntonymsSwitch,
    CorrectCommonMisspellings,
    NeuralParaphaserPerturbation, # somewhat slow (4.03s/ba)
    SentenceSubjectObjectSwitch,
    # NumericToWord, # InvalidOperation: [<class 'decimal.ConversionSyntax'>]
    MultilingualDictionaryBasedCodeSwitch,
    # DiverseParaphrase, # runtime too long (does not attempt to use gpu)
    EnglishInflectionalVariation,
    ReplaceNumericalValues,
    SpeechDisfluencyPerturbation,
    WhitespacePerturbation,
    ContractionExpansions,
    ColorTransformation,
    # ContextualMeaningPerturbation, # runtime too long (not gpu compatible)
    PigLatin,
    LeetLetters,
    # HashtagifyTransformation, # error: nothing to repeat at position 0
    # CheckSrl, # RuntimeError: The size of tensor a (1127) must match the size of tensor b (512) at non-singleton dimension 1
    # GeoNamesTransformation, # does not return anything
]
transforms = [t for t in nlaug_transforms]
transforms = sorted(transforms, key=lambda t: t.__name__)
transforms = [Transform(t, task_name=task_name) for t in transforms]

initializing Abbreviate
setting max_outputs=1
initializing AbbreviationInsertionEN
setting max_outputs=1
initializing AmericanizeBritishizeEnglish
setting max_outputs=1
initializing AntonymsSubstitute


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fabri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


setting max_outputs=1
initializing AzertyQwertyCharsSwap
setting max_outputs=1
initializing BackTranslation


setting max_outputs=1
initializing ButterFingersPerturbation
setting max_outputs=1
initializing ChangeCharCase
setting max_outputs=1
initializing ChangeDateFormat
setting max_outputs=1
initializing CityNamesTransformation
setting max_outputs=1
initializing ColorTransformation
setting max_outputs=1
initializing ConcatMonolingual
setting max_outputs=1
initializing ContractionExpansions
setting max_outputs=1
initializing CorrectCommonMisspellings
setting max_outputs=1
initializing CountryStateAbbreviation
setting max_outputs=1
initializing DiacriticRemoval
setting max_outputs=1
initializing DiscourseMarkerSubstitution
setting max_outputs=1
initializing DyslexiaWordsSwap
setting max_outputs=1
initializing EnglishInflectionalVariation
setting max_outputs=1
initializing FillerWordAugmentation
setting max_outputs=1
initializing GenderSwap
Loading Operation GenderSwap
setting max_outputs=1
initializing GreetingsAndFarewells
setting max_outputs=1
initializing HashtagGeneration
setting max_outpu

setting max_outputs=1
initializing MultilingualDictionaryBasedCodeSwitch
setting max_outputs=1
initializing NeuralParaphaserPerturbation


...

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fabri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


setting max_outputs=1
initializing SynonymSubstitution


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fabri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


setting max_outputs=1
initializing TenseTransformation with to_tense='past'
setting max_outputs=1
initializing TokenReplacement
setting max_outputs=1
initializing TransformerFill


setting max_outputs=1
initializing UnitConverter
setting max_outputs=1
initializing UseAcronyms
setting max_outputs=1
initializing VisualAttackLetters
setting max_outputs=1
initializing WeekdayMonthAbbreviation
setting max_outputs=1
initializing WhitespacePerturbation
setting max_outputs=1
initializing YesNoQuestionPerturbation
setting max_outputs=1
initializing YodaPerturbation
setting max_outputs=1


In [19]:
scorers = [
    (Likelihood(), "Likelihood"),
    (InverseLikelihood(), "InverseLikelihood"),
    (LikelihoodShift(dataset, direction="positive"), "LikelihoodShiftPos"),
    (LikelihoodShift(dataset, direction="negative"), "LikelihoodShiftNeg"),
    (CleanLabSafe(), "CleanLabSafe"),
]

aggregations = ["sum", "avg"]

In [23]:
dataset_config = ("glue", "sst2")
task_name = "sentiment"

for agg in aggregations:
    for scorer, scorer_name in scorers:

        print(f"running fada-v2 policy search using {scorer_name}-{agg}")

        dataset = load_dataset(*dataset_config, split="train")
        dataset = dataset.rename_column("sentence", "text")

        # initialize extractors
        feature_extractor = AMRFeatureExtractor()
        perf_extractor = PerformanceExtractor(dataset.builder_name, Likelihood())

        # initialize dataset + annotations
        if os.path.exists("./datasets/glue.sst2.annotated"):
            dataset = load_from_disk("./datasets/glue.sst2.annotated")
            features = np.array(dataset["features"])
        else:
            features = feature_extractor(dataset["text"])
            preds = perf_extractor.extract_prediction_probabilities(dataset["text"])
            dataset = dataset.add_column("features", [f for f in features])
            dataset = dataset.add_column("preds", [p.numpy() for p in preds])
            dataset.save_to_disk("./datasets/glue.sst2.annotated")

        # initialize save directory
        save_dir       = f"./fadata/nlaug/v2-{scorer_name}-{agg}/"
        os.makedirs(save_dir, exist_ok=True)

        # initialize fadata arrays
        num_rows       = len(dataset)
        num_transforms = len(transforms)
        num_features   = len(feature_extractor.featurizers)

        performances   = np.zeros((num_transforms, num_features))
        counts         = np.zeros((num_transforms, num_features))
        changes        = np.zeros((num_transforms, num_features))
        policy         = np.full((num_transforms, num_features), fill_value=1/num_transforms)

        min_coverage = 128
        num_to_transform_per_step = 32

        policy_difference = np.inf
        convergence_threshold = 1 / (num_transforms + num_features)

        # run fada-v2 convergence loop

        i = 0
        while policy_difference > convergence_threshold:

            # find low coverage (t,f) pairs
            ts, fs   = np.where(changes < min_coverage)
            tf_pairs = list(zip(ts, fs))

            for t, f in tqdm(tf_pairs):

                f_candidates  = np.where(features[:,f] == 1)[0]

                # feature missing in dataset
                if not f_candidates.size:
                    continue

                num_to_sample = num_to_transform_per_step if len(f_candidates) > num_to_transform_per_step else len(f_candidates)
                f_indices     = np.random.choice(f_candidates, num_to_sample, replace=False)
                f_dataset     = dataset.select(f_indices)

                t_prob = np.zeros(num_transforms)
                t_prob[t] = 1
                transform_probabilities = np.array([t_prob for _ in range(f_dataset.num_rows)])

                augmenter = Augmenter(dataset=f_dataset, 
                              transforms=transforms,  
                              transform_probabilities=transform_probabilities,
                              num_augmentations_per_record=1,
                              num_transforms_to_apply=1,
                              batch_size=10, 
                              keep_originals=False)
                aug_dataset = augmenter.augment()

                try:
                    performance = perf_extractor(aug_dataset["text"], aug_dataset["label"], indices=f_indices)
                except Exception as e:
                    print(e)
                    performance = [1] * f_dataset.num_rows
                aug_dataset = aug_dataset.add_column("performance", [p for p in performance])

                performance = np.array(aug_dataset["performance"]).sum()
                num_changes = np.array(aug_dataset["is_changed"]).sum()

                counts[t,f]       += f_dataset.num_rows
                changes[t,f]      += num_changes  
                performances[t,f] += performance   

            # compute augmentation policy
            if agg == "sum":
                aggregated_performance = performances 
            elif agg == "avg":
                aggregated_performance = np.nan_to_num(performances / counts, 0)
            applicability_rate         = np.nan_to_num(changes / counts, 0)
            new_policy                 = softmax(aggregated_performance * applicability_rate, axis=0)

            policy_difference          = np.linalg.norm(new_policy - policy)
            policy                     = new_policy

            print(f"policy_difference: {policy_difference} (convergence_threshold: {convergence_threshold})")

            policy_heatmap(policy, transforms, feature_extractor.featurizers)

            print("Saving intermediate matrices...")
            np.save(os.path.join(save_dir, f"glue.sst2.fada.v2.counts-step-{i}"), counts)
            np.save(os.path.join(save_dir, f"glue.sst2.fada.v2.changes-step-{i}"), changes)
            np.save(os.path.join(save_dir, f"glue.sst2.fada.v2.performances-step-{i}"), performances)
            np.save(os.path.join(save_dir, f"glue.sst2.fada.v2.policy-step-{i}"), policy)

            i += 1

        # augment original dataset with newly converged policy
        dataset = load_dataset(*dataset_config, split="train")
        dataset = dataset.rename_column("sentence", "text")

        policy_probabilities = implement_policy_probabilities(policy, features)

        augmenter = Augmenter(dataset=dataset, 
                              transforms=transforms, 
                              transform_probabilities=policy_probabilities, 
                              num_augmentations_per_record=1,
                              num_transforms_to_apply=1,
                              keep_originals=True)
        aug_dataset = augmenter.augment()
        aug_dataset.save_to_disk(f"./datasets/glue.sst2.nlaug.fada_v2_{scorer_name}_{agg}")

running fada-v2 policy search using Likelihood


2023-02-21 20:00:48,272 | datasets.builder | WARNING | Found cached dataset glue (C:/Users/fabri/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Using distilbert-base-uncased-finetuned-sst-2-english to support evaluation.


  0%|          | 0/2376 [00:00<?, ?it/s]

KeyboardInterrupt: 